## We solve the MHD toy model with PINN

### The evolution equations are:

$$
\partial_t B^i = \nabla_j (v^j B^i - v^i B^j) \;\;\;\; \nabla_i B^i = 0
$$

The vector field $v^i$ is given. 

And we impose Dirichlet boundary conditions.

The initial data is: 

\begin{align*}
B_1 &= \partial_y \phi \\
B_2 &= -\partial_x \phi
\end{align*}
with (here we use xmin=0, xmax = 1, same for y)

$$
\phi(x,y) = (x*(x-1)*y*(y-1))^2
$$

The velocity field is time-independent and given by:

\begin{align*}
v1(t,x,y) &= \sin(\pi*x)*\cos(\pi*y) \\
v2(t,x,y) &= \cos(\pi*x)*\sin(\pi*y) \\
\end{align*}



In [ ]:
#import Pkg
#Pkg.add(url="https://github.com/psy3nt1st/Optim.jl.git")
using Optim
#using NeuralPDE
#using Optimization
using OptimizationOptimJL
using Roots
using LineSearches
using ModelingToolkit, IntervalSets 
using IntervalSets
using Plots, Printf
using Lux, LuxCUDA, ComponentArrays, Random
using JLD2, LinearAlgebra 
using Revise
using Distributions
using Zygote

const gpud = gpu_device()

includet("../neural_tools.jl")

In [ ]:
@load "toy_MHD_2D_Man.jld2" config Θ st losses

NN = create_chain(config)

In [ ]:
plot(losses)

In [ ]:
@unpack xmin, xmax, ymin, ymax, tmin, tmax = config
xs = collect(xmin:0.01:xmax)
ys = collect(ymin:0.01:ymax);


In [ ]:
t = 0.0
B1_approx = [NN(vcat(t, x, y), Θ, st)[1][1] for x in xs, y in ys]
B2_approx = [NN(vcat(t, x, y), Θ, st)[1][2] for x in xs, y in ys]
p1 = plot(ys, xs, B1_approx, 
        linetype = :contourf, 
        #st = :surface,
        title = "predict B1", aspect_ratio = 1)
p2 = plot(ys, xs, B2_approx, 
        linetype = :contourf, 
        #st = :surface,
        title = "predict B2", aspect_ratio = 1)
ps = [p1, p2]
plot(ps..., layout = (1,2), size = (900,400))
